# Particle-Laden Flow Example

This notebook demonstrates particle-laden flow simulations in PHASTA, including particle tracking, collisions, and fluid-particle interactions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from phasta import FlowSolver, FlowConfig, Mesh, ParticleConfig

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')

## Problem Setup

We'll simulate particles in a channel flow with the following parameters:
- Channel dimensions: 1m × 0.1m
- Particle diameter: 0.001m
- Particle density: 2500 kg/m³
- Number of particles: 1000
- Inlet velocity: 1 m/s
- Reynolds number: 1000

In [ ]:
# Create base configuration
config = FlowConfig()
config.domain = {
    'width': 1.0,
    'height': 0.1,
    'mesh_size': 0.005
}

config.flow = {
    'inlet_velocity': 1.0,
    'reynolds_number': 1000,
    'fluid_density': 1000.0,
    'fluid_viscosity': 0.001,
    'time_step': 0.001,
    'max_time': 1.0
}

config.boundary_conditions = {
    'inlet': 'velocity',
    'outlet': 'outflow',
    'top_wall': 'no-slip',
    'bottom_wall': 'no-slip'
}

## Particle-Laden Channel Flow

Let's simulate particles in a channel flow using the Eulerian-Lagrangian approach.

In [ ]:
# Create mesh
mesh = Mesh.generate_channel_2d(
    width=config.domain['width'],
    height=config.domain['height'],
    nx=200,
    ny=20
)

# Plot mesh
plt.figure(figsize=(12, 2))
mesh.plot()
plt.title('Channel Mesh')
plt.show()

In [ ]:
# Configure particles
particle_config = ParticleConfig()
particle_config.number_of_particles = 1000
particle_config.particle_properties = {
    'diameter': 0.001,
    'density': 2500.0,
    'restitution': 0.8,
    'friction': 0.3
}
particle_config.initial_conditions = {
    'distribution': 'random',
    'region': {
        'x_min': 0.0,
        'x_max': 0.1,
        'y_min': 0.0,
        'y_max': 0.1
    },
    'velocity': [1.0, 0.0]
}
particle_config.coupling = {
    'two_way': True,
    'drag_model': 'schiller_naumann',
    'lift_model': 'saffman_mei'
}
particle_config.collisions = {
    'enabled': True,
    'model': 'soft_sphere',
    'search_method': 'grid'
}

# Create and run solver
solver = FlowSolver(config, mesh, particle_config=particle_config)
results = solver.solve()

print("\nParticle-Laden Flow Results:")
print(f"Simulation completed: {results.completed}")
print(f"Final time: {results.final_time}")
print(f"Number of time steps: {results.time_steps}")

## Visualize Results

Plot particle positions, velocity field, and particle statistics.

In [ ]:
# Plot results at different times
times = [0.0, 0.25, 0.5, 0.75, 1.0]
plt.figure(figsize=(15, 10))

for i, t in enumerate(times):
    plt.subplot(2, 3, i+1)
    results.plot_particle_positions(t)
    plt.title(f'Particle Positions at t = {t}s')
    plt.colorbar(label='Particle Diameter (m)')

plt.tight_layout()
plt.show()

In [ ]:
# Plot velocity field and particle statistics
plt.figure(figsize=(15, 5))

# Plot velocity field
plt.subplot(131)
results.plot_velocity_magnitude(0.5)
plt.title('Velocity Field at t = 0.5s')
plt.colorbar(label='Velocity (m/s)')

# Plot particle concentration
plt.subplot(132)
results.plot_particle_concentration(0.5)
plt.title('Particle Concentration at t = 0.5s')
plt.colorbar(label='Concentration (particles/m³)')

# Plot particle velocity distribution
plt.subplot(133)
results.plot_particle_velocity_distribution(0.5)
plt.title('Particle Velocity Distribution at t = 0.5s')
plt.colorbar(label='Velocity (m/s)')

plt.tight_layout()
plt.show()

## Particle Collision Analysis

Now let's analyze particle collisions and their effects.

In [ ]:
# Plot collision statistics
plt.figure(figsize=(15, 5))

# Plot collision frequency
plt.subplot(131)
results.plot_collision_frequency()
plt.title('Collision Frequency')
plt.xlabel('Time (s)')
plt.ylabel('Collisions/s')
plt.grid(True)

# Plot collision energy
plt.subplot(132)
results.plot_collision_energy()
plt.title('Collision Energy')
plt.xlabel('Time (s)')
plt.ylabel('Energy (J)')
plt.grid(True)

# Plot particle size distribution
plt.subplot(133)
results.plot_particle_size_distribution()
plt.title('Particle Size Distribution')
plt.xlabel('Diameter (m)')
plt.ylabel('Number of Particles')
plt.grid(True)

plt.tight_layout()
plt.show()

## Particle-Wall Interaction

Let's analyze particle-wall interactions and deposition.

In [ ]:
# Plot wall interaction statistics
plt.figure(figsize=(15, 5))

# Plot wall collision frequency
plt.subplot(131)
results.plot_wall_collision_frequency()
plt.title('Wall Collision Frequency')
plt.xlabel('Time (s)')
plt.ylabel('Collisions/s')
plt.grid(True)

# Plot particle deposition
plt.subplot(132)
results.plot_particle_deposition()
plt.title('Particle Deposition')
plt.xlabel('Position (m)')
plt.ylabel('Deposited Mass (kg)')
plt.grid(True)

# Plot wall shear stress
plt.subplot(133)
results.plot_wall_shear_stress()
plt.title('Wall Shear Stress')
plt.xlabel('Position (m)')
plt.ylabel('Shear Stress (Pa)')
plt.grid(True)

plt.tight_layout()
plt.show()

## Exercises

1. Try different particle sizes and densities
2. Experiment with different collision models
3. Study the effect of particle shape on flow
4. Compare different drag and lift models

## Next Steps

- Try the phase change example
- Explore turbulent particle-laden flows
- Learn about particle agglomeration